In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# creating word dictionary
def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]    
    all_words = []       
    for mail in emails:
        filename, file_extension = os.path.splitext(mail)
        if file_extension != ".txt":
            continue
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words

    dictionary = Counter(all_words)
    list_to_remove = [*dictionary]
    
    for item in list_to_remove:
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
        # dictionary = dictionary.most_common(3000)
    return dictionary
 

In [3]:
#feature extraction process
def extract_features(mail_dir): 
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    print(len(files))
    features_matrix = np.zeros((len(files),17000))
    docID = 0;
    for fil in files:
      filename, file_extension = os.path.splitext(fil)
      if file_extension != ".txt":
            continue
      with open(fil) as fi:
        for i,line in enumerate(fi):
          if i == 2:
            words = line.split()
            for word in words:
              wordID = 0
              for i,d in enumerate(dictionary):
                if d[0] == word:
                  wordID = i
                  # print("{} - {}".format(docID, wordID))
                  # print("---------------")
                  features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1     
    return features_matrix

In [ ]:
# training the classifiers
import os
import numpy as np
from collections import Counter
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
 
# Create a dictionary of words with its frequency

 
train_dir = '/content/drive/My Drive/Colab Notebooks/York/ds1/data/train-mails/'
dictionary = make_Dictionary(train_dir)
 
# Prepare feature vectors per training mail and its labels
 
train_labels = np.zeros(702)
train_labels[351:701] = 1
train_matrix = extract_features(train_dir)
 
# Training SVM and Naive bayes classifier
 
model1 = MultinomialNB()
model2 = LinearSVC()
model1.fit(train_matrix,train_labels)
model2.fit(train_matrix,train_labels)
 
# Test the unseen mails for Spam
test_dir = '/content/drive/My Drive/Colab Notebooks/York/ds1/data/test-mails/'
test_matrix = extract_features(test_dir)
test_labels = np.zeros(260)
test_labels[130:260] = 1
result1 = model1.predict(test_matrix)
result2 = model2.predict(test_matrix)
print(confusion_matrix(test_labels,result1))
print(confusion_matrix(test_labels,result2))

703
